### 官方案例
https://pytorch.org/docs/master/onnx.html#example-end-to-end-alexnet-from-pytorch-to-onnx   
## Example: End-to-end AlexNet from PyTorch to ONNX
Here is a simple script which exports a pretrained AlexNet as defined in torchvision into ONNX. It runs a single round of inference and then saves the resulting traced model to alexnet.onnx:

In [1]:
import torch
import torchvision

dummy_input = torch.randn(10, 3, 224, 224)
model = torchvision.models.alexnet(pretrained=True)

# Providing input and output names sets the display names for values
# within the model's graph. Setting these does not change the semantics
# of the graph; it is only for readability.
#
# The inputs to the network consist of the flat list of inputs (i.e.
# the values you would pass to the forward() method) followed by the
# flat list of parameters. You can partially specify names, i.e. provide
# a list here shorter than the number of inputs to the model, and we will
# only set that subset of names, starting from the beginning.
input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
output_names = [ "output1" ]

torch.onnx.export(model, dummy_input, "alexnet.onnx", verbose=True, input_names=input_names, output_names=output_names)

graph(%actual_input_1 : Float(10, 3, 224, 224)
      %learned_0 : Float(64, 3, 11, 11)
      %learned_1 : Float(64)
      %learned_2 : Float(192, 64, 5, 5)
      %learned_3 : Float(192)
      %learned_4 : Float(384, 192, 3, 3)
      %learned_5 : Float(384)
      %learned_6 : Float(256, 384, 3, 3)
      %learned_7 : Float(256)
      %learned_8 : Float(256, 256, 3, 3)
      %learned_9 : Float(256)
      %learned_10 : Float(4096, 9216)
      %learned_11 : Float(4096)
      %learned_12 : Float(4096, 4096)
      %learned_13 : Float(4096)
      %learned_14 : Float(1000, 4096)
      %learned_15 : Float(1000)) {
  %17 : Float(10, 64, 55, 55) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[11, 11], pads=[2, 2, 2, 2], strides=[4, 4]](%actual_input_1, %learned_0, %learned_1), scope: AlexNet/Sequential[features]/Conv2d[0]
  %18 : Float(10, 64, 55, 55) = onnx::Relu(%17), scope: AlexNet/Sequential[features]/ReLU[1]
  %19 : Float(10, 64, 27, 27) = onnx::MaxPool[kernel_shape=[3, 3], pads=[0, 0, 

The resulting alexnet.onnx is a binary protobuf file which contains both the network structure and parameters of the model you exported (in this case, AlexNet). The keyword argument verbose=True causes the exporter to print out a human-readable representation of the network:

You can also verify the protobuf using the ONNX library. You can install ONNX with conda:    
conda install -c conda-forge onnx   

Then, you can run:

In [1]:
import onnx

# Load the ONNX model
model = onnx.load("alexnet.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
onnx.helper.printable_graph(model.graph)


'graph torch-jit-export (\n  %actual_input_1[FLOAT, 10x3x224x224]\n) optional inputs with matching initializers (\n  %learned_0[FLOAT, 64x3x11x11]\n  %learned_1[FLOAT, 64]\n  %learned_2[FLOAT, 192x64x5x5]\n  %learned_3[FLOAT, 192]\n  %learned_4[FLOAT, 384x192x3x3]\n  %learned_5[FLOAT, 384]\n  %learned_6[FLOAT, 256x384x3x3]\n  %learned_7[FLOAT, 256]\n  %learned_8[FLOAT, 256x256x3x3]\n  %learned_9[FLOAT, 256]\n  %learned_10[FLOAT, 4096x9216]\n  %learned_11[FLOAT, 4096]\n  %learned_12[FLOAT, 4096x4096]\n  %learned_13[FLOAT, 4096]\n  %learned_14[FLOAT, 1000x4096]\n  %learned_15[FLOAT, 1000]\n) {\n  %17 = Conv[dilations = [1, 1], group = 1, kernel_shape = [11, 11], pads = [2, 2, 2, 2], strides = [4, 4]](%actual_input_1, %learned_0, %learned_1)\n  %18 = Relu(%17)\n  %19 = MaxPool[kernel_shape = [3, 3], pads = [0, 0, 0, 0], strides = [2, 2]](%18)\n  %20 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%19, %learned_2, %learned_3)\n  %21 = Rel

You can also run the exported model with ONNX Runtime, you will need to install ONNX Runtime: please follow these instructions.

Once these are installed, you can use the backend for ONNX Runtime:

In [5]:
# ...continuing from above
import onnxruntime as ort
import numpy as np

ort_session = ort.InferenceSession('alexnet.onnx')

outputs = ort_session.run(None, {'actual_input_1': np.random.randn(10, 3, 224, 224).astype(np.float32)})

print(outputs[0])

[[ 2.8922364e-01 -1.4583788e+00 -1.1044384e+00 ... -1.3596333e+00
  -8.9589554e-01  8.7897116e-01]
 [ 4.7992855e-02 -1.3752857e+00 -1.5788202e+00 ... -1.3869075e+00
  -1.3458562e+00  1.3426404e+00]
 [ 1.0668859e-03 -1.2775283e+00 -1.3748541e+00 ... -1.0736375e+00
  -1.1849667e+00  1.2001125e+00]
 ...
 [ 8.2895130e-02 -1.6536072e+00 -2.3187499e+00 ... -1.6493472e+00
  -9.1081142e-01  8.1337160e-01]
 [ 2.5782725e-01 -1.5074546e+00 -1.0070571e+00 ... -9.0397274e-01
  -1.1938461e+00  1.3289818e+00]
 [ 6.3743442e-03 -1.3554738e+00 -1.1884270e+00 ... -8.5809034e-01
  -1.1055359e+00  1.1084468e+00]]
